In [1]:
!pip install telebot

In [2]:
!pip install roboflow supervision opencv-python

   ---------------------------------------- 0.0/76.7 kB ? eta -:--:--
   ---------------- ----------------------- 30.7/76.7 kB ? eta -:--:--
   ---------------- ----------------------- 30.7/76.7 kB ? eta -:--:--
   ---------------- ----------------------- 30.7/76.7 kB ? eta -:--:--
   ---------------- ----------------------- 30.7/76.7 kB ? eta -:--:--
   --------------------- ------------------ 41.0/76.7 kB 131.3 kB/s eta 0:00:01
   --------------------- ------------------ 41.0/76.7 kB 131.3 kB/s eta 0:00:01
   --------------------- ------------------ 41.0/76.7 kB 131.3 kB/s eta 0:00:01
   --------------------- ------------------ 41.0/76.7 kB 131.3 kB/s eta 0:00:01
   --------------------- ------------------ 41.0/76.7 kB 131.3 kB/s eta 0:00:01
   --------------------- ------------------ 41.0/76.7 kB 131.3 kB/s eta 0:00:01
   --------------------- ------------------ 41.0/76.7 kB 131.3 kB/s eta 0:00:01
   --------------------- ------------------ 41.0/76.7 kB 131.3 kB/s eta 0:00:01
   ---

In [5]:
!pip install inference

     ---------------------------------------- 0.0/125.2 kB ? eta -:--:--
     ------------ -------------------------- 41.0/125.2 kB 1.9 MB/s eta 0:00:01
     -------------------------------------- 125.2/125.2 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/49.3 kB ? eta -:--:--
     ---------------------------------------- 49.3/49.3 kB 2.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/532.4 kB ? eta -:--:--
   --------- ------------------------------ 122.9/532.4 kB 7.0 MB/s eta 0:00:01
   ------------------- -------------------- 256.0/532.4 kB 3.9 MB/s eta 0:00:01
   ---------------------------------------  522.2/532.4 kB 3.6 MB/s eta 0:00:01
   ---------------------------------------- 532.4/532.4 kB 3.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/148.1 kB ? eta -:--:--
   -------------------------------------- - 143.4/1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.7.0 requires botocore<1.31.65,>=1.31.16, but you have botocore 1.34.145 which is incompatible.
anaconda-cloud-auth 0.1.4 requires pydantic<2.0, but you have pydantic 2.8.2 which is incompatible.
df2gspread 1.0.4 requires google-api-python-client==1.6.7, but you have google-api-python-client 2.137.0 which is incompatible.
pylint 2.16.2 requires colorama>=0.4.5; sys_platform == "win32", but you have colorama 0.4.4 which is incompatible.


  Using cached inference-0.15.1-py3-none-any.whl.metadata (14 kB)
  Using cached APScheduler-3.10.1-py3-none-any.whl.metadata (5.7 kB)
  Using cached Cython-3.0.0-cp311-cp311-win_amd64.whl.metadata (3.2 kB)
  Using cached fastapi-0.110.3-py3-none-any.whl.metadata (24 kB)
  Using cached opencv_python-4.8.0.76-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached piexif-1.1.3-py2.py3-none-any.whl.metadata (3.7 kB)
  Using cached prometheus_fastapi_instrumentator-6.0.0-py3-none-any.whl.metadata (13 kB)
  Using cached redis-5.0.7-py3-none-any.whl.metadata (9.3 kB)
  Using cached pybase64-1.3.2-cp311-cp311-win_amd64.whl.metadata (8.2 kB)
  Using cached pytest_asyncio-0.21.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached pydantic-2.8.2-py3-none-any.whl.metadata (125 kB)
  Using cached pydantic_settings-2.3.4-py3-none-any.whl.metadata (3.3 kB)
  Using cached openai-1.37.0-py3-none-any.whl.metadata (22 kB)
  Using cached structlog-24.4.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached z

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.7.0 requires botocore<1.31.65,>=1.31.16, but you have botocore 1.34.145 which is incompatible.
anaconda-cloud-auth 0.1.4 requires pydantic<2.0, but you have pydantic 2.8.2 which is incompatible.
df2gspread 1.0.4 requires google-api-python-client==1.6.7, but you have google-api-python-client 2.137.0 which is incompatible.
pylint 2.16.2 requires colorama>=0.4.5; sys_platform == "win32", but you have colorama 0.4.4 which is incompatible.


In [ ]:
# Импортируем необходимые библиотеки
import telebot
import requests
from inference import get_model
import supervision as sv
import cv2

# Импортируем ключи из конфигурационного файла
from config import API_TOKEN, api_key

bot = telebot.TeleBot(API_TOKEN)

# Handle '/start' and '/help'
@bot.message_handler(commands=['help', 'start'])
def send_welcome(message):
    bot.reply_to(message, """\
Hi there, I am EchoBot.
I am here to echo your kind words back to you. Just say anything nice and I'll say the exact same thing to you!\
""")

# Handle all other messages with content_type 'photo'
@bot.message_handler(content_types=['photo'])
def handle_photo(message):
    # Сохраняем полученное фото
    photo = message.photo[-1]
    file_info = bot.get_file(photo.file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    save_path = 'photo.jpg'
    with open(save_path, 'wb') as new_file:
        new_file.write(downloaded_file)
        bot.reply_to(message, 'Фотография сохранена.')
    
    # Загружаем фото для обработки
    image_file = "photo.jpg"
    image = cv2.imread(image_file)

    # Выполняем анализ с помощью модели и получаем разметку
    model = get_model(model_id="car-damage-coco-dataset/4", api_key=api_key)  # исправлено здесь
    results = model.infer(image)[0]
    detections = sv.Detections.from_inference(results)
    print(detections)

    # Аннотируем изображение с помощью supervision
    bounding_box_annotator = sv.BoundingBoxAnnotator()
    label_annotator = sv.LabelAnnotator()
    annotated_image = bounding_box_annotator.annotate(scene=image, detections=detections)
    annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections)

    sv.plot_image(annotated_image)

    # Сохраняем аннотированное изображение
    annotated_image_path = 'annotated_photo.jpg'
    cv2.imwrite(annotated_image_path, annotated_image)

    # Отправляем аннотированное изображение пользователю
    with open(annotated_image_path, 'rb') as photo:
        bot.send_photo(message.chat.id, photo)

    # Отвечаем пользователю
    bot.reply_to(message, 'Фотография с разметкой отправлена.')

# Запускаем бота
bot.infinity_polling()